In [14]:
import logging
import argparse
import csv
import configparser
from TM1py.Services import TM1Service
from TM1py.Objects import Dimension, Hierarchy, Element,Cube,ElementAttribute
from collections import defaultdict
from TM1py.Utils import CaseAndSpaceInsensitiveTuplesDict
import datetime
import json
import pandas as pd
import mdxpy

In [2]:
TM1_CLOUD_PARAMS = {
    "base_url": "https://cubewise.planning-analytics.ibmcloud.com/tm1/api/plansamp",
    "user": "cubewise01_tm1_automation",
    "namespace": "LDAP",
    "password": "XI2b5FryLHR679",
    "ssl": True,
    "verify": True,
    "async_requests_mode": True
}

In [41]:
def get_tm1_service(instance_name):
    config = configparser.ConfigParser()
    config.read(r'config.ini')
    return TM1Service(**config[instance_name])

In [4]:
def read_json_file_to_dataframe(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        df = pd.json_normalize(data)
        return df,data

In [5]:
df_sales,dict_data_sales = read_json_file_to_dataframe(r'C:\Users\User\tm1py_demo\其他\sales.json')

In [6]:
df_sales

,Region,Month,Customer,ProductID,SalesChannel,Measure,Value
0,New Zealand,2022-01,Williams Tech,P002,Retail,Sales Quantity,283
1,New Zealand,2022-01,Williams Tech,P004,Online,Sales Quantity,62
2,New Zealand,2022-01,Williams Tech,P006,Online,Sales Quantity,50
3,New Zealand,2022-01,Williams Tech,P008,Online,Sales Quantity,476
4,New Zealand,2022-01,Williams Tech,P013,Retail,Sales Quantity,306
...,...,...,...,...,...,...,...
263776,Venezuela,2023-12,Anderson Ventures,P093,Online,Sales Quantity,5
263777,Venezuela,2023-12,Anderson Ventures,P095,Wholesale,Sales Quantity,195
263778,Venezuela,2023-12,Anderson Ventures,P097,Wholesale,Sales Quantity,389
263779,Venezuela,2023-12,Anderson Ventures,P099,Retail,Sales Quantity,128


In [7]:
df_sales['Version']='Actual'
df_sales=df_sales[['Month','Version','Region','Customer','ProductID','SalesChannel','Measure','Value']]

In [52]:
tm1_cloud = TM1Service(**TM1_CLOUD_PARAMS)
print(tm1_cloud.server.get_server_name())

plansamp


In [42]:
tm1 = get_tm1_service('Hello_World')

In [44]:
print(tm1.server.get_server_name())
mdx='''
        SELECT 
          NON EMPTY 
           {[Sales Measure].[Sales Measure].[Sales Quantity]} 
          ON COLUMNS , 
          NON EMPTY 
           {TM1FILTERBYLEVEL({TM1SUBSETALL([Product].[Product])}, 0)}
           * {TM1FILTERBYLEVEL({[Period].[Period].[2023],[Period].[Period].[2023-01],[Period].[Period].[2023-02],[Period].[Period].[2023-03],[Period].[Period].[2023-04],[Period].[Period].[2023-05],[Period].[Period].[2023-06],[Period].[Period].[2023-07],[Period].[Period].[2023-08],[Period].[Period].[2023-09],[Period].[Period].[2023-10],[Period].[Period].[2023-11],[Period].[Period].[2023-12]}, 0)}
           * {TM1FILTERBYLEVEL({TM1SUBSETALL([Region].[Region])}, 0)}
           * {TM1FILTERBYLEVEL({TM1SUBSETALL([Customer].[Customer])}, 0)}
           * {TM1FILTERBYLEVEL({TM1SUBSETALL([Sales Channel].[Sales Channel])}, 0)} 
          ON ROWS 
        FROM [Sales] 
        WHERE 
          (
           [Version].[Version].[Actual]
          )
    
      '''
df_sales_budget=tm1.cells.execute_mdx_dataframe(mdx=mdx)

Hello_World


In [45]:
df_sales_budget

,Product,Period,Region,Customer,Sales Channel,Sales Measure,Value
0,P001,2023-01,United States,Williams Tech,Retail,Sales Quantity,272.0
1,P001,2023-01,Canada,Harris Group,Retail,Sales Quantity,342.0
2,P001,2023-01,Canada,Rivera Partners,Retail,Sales Quantity,426.0
3,P001,2023-01,Mexico,Garcia Devices,Online,Sales Quantity,147.0
4,P001,2023-01,Mexico,Martin Holdings,Retail,Sales Quantity,11.0
...,...,...,...,...,...,...,...
131909,P100,2023-12,Slovenia,Adams Group,Wholesale,Sales Quantity,263.0
131910,P100,2023-12,Slovakia,Wilson Traders,Retail,Sales Quantity,454.0
131911,P100,2023-12,Slovakia,Scott Enterprises,Wholesale,Sales Quantity,174.0
131912,P100,2023-12,Lithuania,Nguyen Electronics,Online,Sales Quantity,412.0


In [47]:
df_sales_budget['Value']=df_sales_budget['Value']*1.1
df_sales_budget['Version']='Budget Neil'
df_sales_budget["Period"] = df_sales_budget["Period"].str.replace("2023", "2024")
df_sales_budget=df_sales_budget[['Period','Version','Region','Customer','Product','Sales Channel','Sales Measure','Value']]
new_version_element = "Budget Neil"
if not tm1_cloud.elements.exists(dimension_name="Version", hierarchy_name="Version", element_name=new_version_element):
    tm1_cloud.elements.create("Version", "Version", element=Element(new_version_element, "Numeric"))
tm1_cloud.cubes.cells.write_dataframe(cube_name="Sales", data=df_sales_budget, use_blob=True,deactivate_transaction_log=True)

In [48]:
# transfer local tm1 to tm1 cloud
mdx='''
        SELECT 
          NON EMPTY 
           {[Sales Measure].[Sales Measure].[Sales Quantity]} 
          ON COLUMNS , 
          NON EMPTY 
           {TM1FILTERBYLEVEL({TM1SUBSETALL([Product].[Product])}, 0)}
           * {TM1FILTERBYLEVEL({[Period].[Period].[2023],[Period].[Period].[2023-01],[Period].[Period].[2023-02],[Period].[Period].[2023-03],[Period].[Period].[2023-04],[Period].[Period].[2023-05],[Period].[Period].[2023-06],[Period].[Period].[2023-07],[Period].[Period].[2023-08],[Period].[Period].[2023-09],[Period].[Period].[2023-10],[Period].[Period].[2023-11],[Period].[Period].[2023-12]}, 0)}
           * {TM1FILTERBYLEVEL({TM1SUBSETALL([Region].[Region])}, 0)}
           * {[Customer].[Customer].[All Customers]}
           * {TM1FILTERBYLEVEL({TM1SUBSETALL([Sales Channel].[Sales Channel])}, 0)} 
          ON ROWS 
        FROM [Sales] 
        WHERE 
          (
           [Version].[Version].[Actual]
          )
    
      '''
df_sales_aggerate=tm1.cells.execute_mdx_dataframe(mdx=mdx)

In [60]:
df_prepare = df_sales_aggerate.drop('Customer',axis=1)

In [62]:
df_prepare

,Product,Period,Region,Sales Channel,Sales Measure,Value
0,P001,2023-01,United States,Retail,Sales Quantity,272.0
1,P001,2023-01,Canada,Retail,Sales Quantity,768.0
2,P001,2023-01,Mexico,Online,Sales Quantity,147.0
3,P001,2023-01,Mexico,Retail,Sales Quantity,11.0
4,P001,2023-01,Brazil,Retail,Sales Quantity,716.0
...,...,...,...,...,...,...
96102,P100,2023-12,Slovenia,Wholesale,Sales Quantity,425.0
96103,P100,2023-12,Slovakia,Retail,Sales Quantity,454.0
96104,P100,2023-12,Slovakia,Wholesale,Sales Quantity,174.0
96105,P100,2023-12,Lithuania,Online,Sales Quantity,412.0


In [64]:
df_prepare['Version']="Budget Neil"
df_prepare = df_prepare[['Period','Version','Region','Product','Sales Channel','Sales Measure','Value']]

In [65]:
df_prepare

,Period,Version,Region,Product,Sales Channel,Sales Measure,Value
0,2023-01,Budget Neil,United States,P001,Retail,Sales Quantity,272.0
1,2023-01,Budget Neil,Canada,P001,Retail,Sales Quantity,768.0
2,2023-01,Budget Neil,Mexico,P001,Online,Sales Quantity,147.0
3,2023-01,Budget Neil,Mexico,P001,Retail,Sales Quantity,11.0
4,2023-01,Budget Neil,Brazil,P001,Retail,Sales Quantity,716.0
...,...,...,...,...,...,...,...
96102,2023-12,Budget Neil,Slovenia,P100,Wholesale,Sales Quantity,425.0
96103,2023-12,Budget Neil,Slovakia,P100,Retail,Sales Quantity,454.0
96104,2023-12,Budget Neil,Slovakia,P100,Wholesale,Sales Quantity,174.0
96105,2023-12,Budget Neil,Lithuania,P100,Online,Sales Quantity,412.0


In [54]:
c = tm1_cloud.cubes.get('Sales Summary')

In [55]:
c

Cube:{"Name": "Sales Summary", "Dimensions@odata.bind": ["Dimensions('Period')", "Dimensions('Version')", "Dimensions('Region')", "Dimensions('Product')", "Dimensions('Sales Channel')", "Dimensions('Sales Measure')"]}

In [66]:
# data clear
mdx='''
        SELECT 
          NON EMPTY 
           {[Sales Measure].[Sales Measure].[Sales Quantity]} 
          ON COLUMNS , 
          NON EMPTY 
           {TM1FILTERBYLEVEL({TM1SUBSETALL([Product].[Product])}, 0)}
           * {TM1FILTERBYLEVEL({TM1SUBSETALL([Period].[Period])}, 0)}
           * {TM1FILTERBYLEVEL({TM1SUBSETALL([Region].[Region])}, 0)}
           * {TM1FILTERBYLEVEL({TM1SUBSETALL([Sales Channel].[Sales Channel])}, 0)} 
          ON ROWS 
        FROM [Sales] 
        WHERE 
          (
           [Version].[Version].[Actual]
          )
    
      '''
cube_tgt='Sales Summary'
tm1_cloud.cells.clear_with_mdx(cube_tgt,mdx)
tm1_cloud.cubes.cells.write_dataframe(cube_name=cube_tgt, data=df_prepare, use_blob=True,deactivate_transaction_log=True)

In [67]:
# syn dim
dim = 'Product'
df_product_from_tm1=tm1.elements.get_elements_dataframe(dim,dim)


In [70]:
tm1_cloud.hierarchies.update_or_create_hierarchy_from_dataframe(
        dimension_name=dim,
        hierarchy_name=dim,
        df=df_product_from_tm1,
        unwind=True)